# 02 — Feature engineering & modeling (Day 4)


In [ ]:
from pathlib import Path
import pandas as pd

obs_path = Path('data/processed/observations.parquet')
assert obs_path.exists(), 'Run build_dataset first.'
df = pd.read_parquet(obs_path)
df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'], utc=True, errors='coerce')
df.head()


## Infer sampling interval (minutes)

In [ ]:
ts = df['timestamp_utc'].sort_values()
diffs = ts.diff().dt.total_seconds().dropna()
interval_minutes = int(round(diffs.median() / 60.0)) if len(diffs) else 10
interval_minutes = max(1, interval_minutes)
interval_minutes


## Build features + targets

In [ ]:
from traffic_forecast.features import FeatureConfig, make_feature_frame

feat_cfg = FeatureConfig(horizons_min=(15, 30), lags=(1, 2, 3), rolling_windows=(3, 6), drop_na=True)
fdf = make_feature_frame(df, cfg=feat_cfg, interval_minutes=interval_minutes)
fdf.head()


## Train models

In [ ]:
from traffic_forecast.model import TrainConfig, train_models
import json

target_cols = ['y_15', 'y_30']
drop_cols = {'timestamp_utc', 'point_id', 'latitude', 'longitude'} | set(target_cols)
feature_cols = [c for c in fdf.columns if c not in drop_cols]

results = train_models(fdf, feature_cols=feature_cols, cfg=TrainConfig(horizons_min=(15, 30), test_fraction=0.2))
Path('reports').mkdir(parents=True, exist_ok=True)
Path('reports/metrics.json').write_text(json.dumps(results, indent=2), encoding='utf-8')
results
